In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import jieba

# 自訂中文文件集，涵蓋多個主題
documents = [
    # ---- 科技（Tech）----
    "我把舊筆電升級到SSD，開機速度變快，日常寫程式與資料處理更順。",
    "這款手機主打AI拍照與夜景模式，但續航表現與充電速度才是我在意的。",
    "智慧家居用語音控制燈光與冷氣，連網穩定度與延遲影響體驗。",
    "最近在看大型語言模型的微調方法，資料清理與評估指標很重要。",
    "顯示卡價格波動大，跑深度學習時GPU記憶體大小決定能否訓練。",
    "雲端服務提供彈性擴充，但成本控管與安全設定需要規劃。",
    "我在測試向量資料庫做語意搜尋，embedding品質影響檢索效果。",
    "新出的路由器支援Wi-Fi 6E，家裡多裝置同時連線比較不會卡。",

    # ---- 財經（Finance）----
    "台股成交量放大但波動也變高，我會分批進場並設定停損。",
    "央行升息讓房貸利率變高，現金流與風險控管變得更重要。",
    "我比較ETF的費用率與追蹤誤差，長期投資要看總成本。",
    "匯率變動影響出國預算，也會影響進口商品與公司毛利。",
    "財報公布後股價大震盪，市場解讀營收成長與毛利率變化。",
    "通膨數據偏高時，消費支出與物價會影響整體景氣預期。",
    "投資前我會看產業趨勢與公司競爭力，不只看短期消息。",
    "高殖利率不一定安全，還要注意現金股利是否可持續。",

    # ---- 健康（Health）----
    "最近睡眠品質差，我調整作息並減少咖啡因，精神狀態改善。",
    "健檢報告顯示血脂偏高，醫師建議飲食控制並增加有氧運動。",
    "流感季節我會戴口罩、勤洗手，也會考慮接種疫苗降低風險。",
    "久坐工作容易肩頸痠痛，我改用站立辦公並做伸展。",
    "我記錄每天走路步數與心率，觀察運動強度對體能的影響。",
    "飲食太油會影響腸胃，清淡料理搭配高纖蔬菜比較舒服。",
    "壓力大時容易頭痛，我會用冥想與呼吸練習減少緊繃。",
    "規律運動提升免疫力，但過度訓練反而可能造成疲勞與受傷。",

    # ---- 旅遊餐飲（Travel/Food）----
    "這次旅行安排了老街小吃與夜市，排隊時間會影響行程節奏。",
    "我比較住宿價格與交通便利性，靠近捷運站的飯店更省時間。",
    "去海邊玩要注意天氣與防曬，行李也要帶輕便雨具。",
    "咖啡店的手沖風味差異很大，我喜歡果香明顯的淺焙豆。",
    "烘焙麵包要控制發酵時間與溫度，口感與香氣差很多。",
    "餐廳評價不只看分數，也要看口味偏好與服務品質。",
    "安排自由行時我會先規劃景點動線，再決定交通票券方案。",
    "地方特色料理通常使用在地食材，價格合理但很看季節。",

    # ---- 混合主題文件（用來展示「一篇多topic」）----
    "出國旅遊除了美食，匯率變動也會影響預算，我會先換好部分外幣。",
    "我想買新手機但也在考慮把預算拿去投資ETF，兩者的成本效益要比較。",
    "跑步訓練需要運動手錶記錄心率，但產品價格與續航會影響選擇。",
    "長途旅行坐太久腰酸背痛，我會做伸展運動並挑選座位提高舒適度。",
    "雲端訂閱服務看似便宜，但月費累積後成本不低，像投資一樣要算總支出。",
    "夜市小吃很吸引人，但我會注意油脂與熱量，避免影響健康檢查數值。"
]

# 簡單停用詞（教學用：短小即可；正式可換成完整停用詞表）
stopwords = set("""
的 了 在 是 也 和 以及 與 但 更 很 我 你 他 她 它 我們 你們 他們
這 那 這次 最近 主要 內容 可能 等等 比較 影響 需要 會 先 再 我會
""".split())

# 使用 jieba 進行斷詞，將停用詞過濾掉，並將斷詞的結果以空格分隔
li_docs = []
for doc in documents:
    # 斷詞
    words = jieba.lcut(doc)

    # 過濾停用詞
    filtered_words = [word for word in words if word not in stopwords]
    
    # 將過濾後的詞語以空格連接成字串，加入文件列表
    li_docs.append(" ".join(filtered_words))

# 使用 CountVectorizer 進行詞頻統計，並建立詞頻矩陣
vectorizer = CountVectorizer(
    min_df=2, # 最少出現 2 次的詞才納入
    max_df=0.8, # 出現比例超過 80% 的詞不納入
    ngram_range=(1, 2) # 同時考慮一元與二元詞組
)
'''
為什麼 min_df 可以是整數也可以是浮點數？
- 當 min_df 是整數時，表示詞語在文件集中至少出現的次數。例如，min_df=2 表示詞語必須至少在 2 篇文件中出現才會被納入。
- 當 min_df 是浮點數時，表示詞語在文件集中至少出現的比例。例如，min_df=0.8 表示詞語必須至少在 80% 的文件中出現才會被納入。
同理，max_df 也是一樣的道理。
'''

# 建立詞頻矩陣
X = vectorizer.fit_transform(li_docs)

# 使用 LDA 進行主題模型訓練
lda = LatentDirichletAllocation(
    n_components=4, # 設定要找的主題數量，這裡設定為 4 個主題，你可以根據需求調整
    max_iter=100, # 最大迭代次數
    random_state=42, # 設定隨機種子以確保結果可重現
    learning_method="batch" # 使用批次學習方式，LDA 有兩種學習方式：online 和 batch，差異在於 batch 是一次使用所有文件進行更新，而 online 是分批次進行更新
)

# 訓練 LDA 模型
lda.fit(X)

# 取得詞語對應的索引
feature_names = vectorizer.get_feature_names_out()

# 顯示主題關鍵詞 (每個主題顯示前 10 個關鍵詞)
for topic_idx, topic in enumerate(lda.components_):
    # 顯示主題編號，對應 documents 中的主題
    print(f"Topic #{topic_idx + 1}:")

    # 取得該主題中權重最高的前 10 個詞語索引
    '''
    topic.argsort()[-10:][::-1]
    語法說明：
    - topic.argsort()：這會返回一個索引陣列，該陣列表示將 topic 陣列中的元素從小到大排序後的索引位置。
    - [-10:]：這會從排序後的索引陣列中取出最後 10 個索引，這些索引對應的是 topic 陣列中權重最高的 10 個元素。
    - [::-1]：這會將取出的 10 個索引反轉順序，使其從權重最高到最低排列。
    總結來說，這行語法的作用是取得 topic 陣列中權重最高的 10 個詞語的索引，並按照從高到低的順序排列。
    '''
    top_features_indices = topic.argsort()[-10:][::-1]
    
    # 取得對應的詞語
    top_features = [feature_names[i] for i in top_features_indices]
    
    # 取得對應的權重
    top_weights = topic[top_features_indices]
    
    # 顯示關鍵詞與權重
    for i in range(len(top_features)):
        print(f"    {top_features[i]} ({top_weights[i]:.2f})")
    print()

Topic #1:
    資料 (3.25)
    控管 (2.25)
    重要 (2.25)
    速度 (2.25)
    變高 (1.26)
    安全 (1.26)
    利率 (1.26)
    風險 (1.26)
    成本 (1.25)
    雲端 (1.25)

Topic #2:
    訓練 (3.25)
    注意 (2.25)
    決定 (2.25)
    記錄 (2.25)
    心率 (2.25)
    容易 (2.25)
    品質 (2.23)
    波動 (1.25)
    價格 (1.25)
    季節 (1.25)

Topic #3:
    控制 (3.25)
    時間 (2.26)
    夜市 (2.25)
    小吃 (2.25)
    飲食 (2.25)
    料理 (2.25)
    支出 (2.25)
    旅行 (1.26)
    運動 (1.26)
    安排 (1.25)

Topic #4:
    預算 (3.25)
    成本 (2.25)
    變動 (2.25)
    匯率 變動 (2.25)
    匯率 (2.25)
    出國 (2.25)
    etf (2.25)
    公司 (1.27)
    考慮 (1.25)
    伸展 (1.25)

